In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns


: 

In [ ]:
# Function to generate synthetic dataset for heart disease detection
def generate_synthetic_heart_data(num_samples=1000, random_seed=42):
    np.random.seed(random_seed)
    # Generate random features (assuming 13 features for heart disease dataset)
    features = [
        "age", "sex", "chest_pain_type", "resting_blood_pressure", "cholesterol",
        "fasting_blood_sugar", "rest_ecg", "max_heart_rate", "exercise_induced_angina",
        "st_depression", "st_slope", "num_major_vessels", "thalassemia"
    ]
    num_features = len(features)
    X = np.random.randn(num_samples, num_features)
    # Generate random labels (binary classification)
    y = np.random.randint(0, 2, size=num_samples)
    return X, y


: 

In [ ]:
# Function to visualize the synthetic heart disease dataset
def visualize_dataset(X, y, feature_names):
    num_features = X.shape[1]
    num_cols = 3
    num_rows = (num_features + num_cols - 1) // num_cols

    plt.figure(figsize=(15, 10))
    for i in range(num_features):
        plt.subplot(num_rows, num_cols, i+1)
        plt.hist(X[:, i], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
        plt.title(f'{feature_names[i]} Distribution')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.grid(True)

    plt.tight_layout()
    plt.show()

# Generate synthetic heart disease dataset
X_synthetic, y_synthetic = generate_synthetic_heart_data()

# Define feature names
feature_names = [
    "Age", "Sex", "Chest Pain Type", "Resting Blood Pressure", "Cholesterol",
    "Fasting Blood Sugar", "Resting ECG", "Max Heart Rate", "Exercise Induced Angina",
    "ST Depression", "ST Slope", "Num Major Vessels", "Thalassemia"
]

# Visualize the synthetic heart disease dataset
visualize_dataset(X_synthetic, y_synthetic, feature_names)


: 

In [ ]:
# Function to visualize the synthetic heart disease dataset
def visualize_dataset(X, y, feature_names):
    num_samples, num_features = X.shape
    num_cols = 3
    num_rows = (num_features + num_cols - 1) // num_cols

    plt.figure(figsize=(15, 10))
    for i in range(num_features):
        plt.subplot(num_rows, num_cols, i+1)
        plt.hist(X[:, i], bins=20, color='skyblue', edgecolor='black', alpha=0.7)

        # Display input value
        plt.text(0.5, 0.5, f'Mean: {np.mean(X[:, i]):.2f}\nStd: {np.std(X[:, i]):.2f}',
                 horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        plt.title(f'{feature_names[i]} Distribution')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.grid(True)

    plt.tight_layout()
    plt.show()

# Generate synthetic heart disease dataset
X_synthetic, y_synthetic = generate_synthetic_heart_data()

# Define feature names
feature_names = [
    "Age", "Sex", "Chest Pain Type", "Resting Blood Pressure", "Cholesterol",
    "Fasting Blood Sugar", "Resting ECG", "Max Heart Rate", "Exercise Induced Angina",
    "ST Depression", "ST Slope", "Num Major Vessels", "Thalassemia"
]

# Visualize the synthetic heart disease dataset
visualize_dataset(X_synthetic, y_synthetic, feature_names)


: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to visualize the distribution of output labels
def visualize_output_labels(y):
    plt.figure(figsize=(6, 4))
    sns.countplot(x=y)
    plt.title('Distribution of Output Labels')
    plt.xlabel('Labels')
    plt.ylabel('Count')
    plt.show()

# Generate synthetic heart disease dataset
X_synthetic, y_synthetic = generate_synthetic_heart_data()

# Visualize the output labels
visualize_output_labels(y_synthetic)


: 

In [ ]:
# Function to generate synthetic dataset for heart disease detection
def generate_synthetic_heart_data(num_samples=1000, num_features=13, random_seed=42):
    np.random.seed(random_seed)
    # Generate random features (assuming 13 features for heart disease dataset)
    X = np.random.randn(num_samples, num_features)
    # Generate random labels (binary classification)
    y = np.random.randint(0, 2, size=num_samples)
    return X, y

: 

In [ ]:
# Generate synthetic heart disease dataset
X_synthetic, y_synthetic = generate_synthetic_heart_data(num_samples=1000, num_features=13)

# Split the synthetic dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_synthetic, y_synthetic, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': [0.1, 0.01, 0.001, 0.0001],  # Kernel coefficient for 'rbf' kernel
}

# Initialize the SVM classifier
svm_classifier = SVC(kernel='rbf', random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Train and evaluate the model
def train_and_evaluate(model, X_train, y_train, X_test, y_test, device):
    with tf.device(device):
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time

        start_time = time.time()
        y_pred = model.predict(X_test)
        inference_time = time.time() - start_time

        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)

        return train_time, inference_time, accuracy, report['macro avg']['f1-score'], report['macro avg']['precision'], report['macro avg']['recall']

# CPU
train_time_cpu, inference_time_cpu, accuracy_cpu, f1_cpu, precision_cpu, recall_cpu = train_and_evaluate(best_model, X_train, y_train, X_test, y_test, "/CPU:0")

# GPU
train_time_gpu, inference_time_gpu, accuracy_gpu, f1_gpu, precision_gpu, recall_gpu = train_and_evaluate(best_model, X_train, y_train, X_test, y_test, "/GPU:0")

# TPU
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    train_time_tpu, inference_time_tpu, accuracy_tpu, f1_tpu, precision_tpu, recall_tpu = train_and_evaluate(best_model, X_train, y_train, X_test, y_test, resolver.master())
except Exception as e:
    print("TPU Error:", e)
    train_time_tpu, inference_time_tpu, accuracy_tpu, f1_tpu, precision_tpu, recall_tpu = -1, -1, -1, -1, -1, -1

# Print results
print("CPU: Train Time =", train_time_cpu, "s, Inference Time =", inference_time_cpu, "s, Accuracy =", accuracy_cpu, ", F1 Score =", f1_cpu, ", Precision =", precision_cpu, ", Recall =", recall_cpu)
print("GPU: Train Time =", train_time_gpu, "s, Inference Time =", inference_time_gpu, "s, Accuracy =", accuracy_gpu, ", F1 Score =", f1_gpu, ", Precision =", precision_gpu, ", Recall =", recall_gpu)
print("TPU: Train Time =", train_time_tpu, "s, Inference Time =", inference_time_tpu, "s, Accuracy =", accuracy_tpu, ", F1 Score =", f1_tpu, ", Precision =", precision_tpu, ", Recall =", recall_tpu)

# Save the best model
model_filename = "svm_model_best.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)


: 

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# Function to generate synthetic dataset for heart disease detection
def generate_synthetic_heart_data(num_samples=1000, num_features=13, random_seed=42):
    np.random.seed(random_seed)
    # Generate random features (assuming 13 features for heart disease dataset)
    X = np.random.randn(num_samples, num_features)
    # Generate random labels (binary classification)
    y = np.random.randint(0, 2, size=num_samples)
    return X, y

# Generate synthetic heart disease dataset
X_synthetic, y_synthetic = generate_synthetic_heart_data(num_samples=1000, num_features=13)

# Split the synthetic dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_synthetic, y_synthetic, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': [0.1, 0.01, 0.001, 0.0001],  # Kernel coefficient for 'rbf' kernel
}

# Initialize the SVM classifier
svm_classifier = SVC(kernel='rbf', random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Train and evaluate the model
def train_and_evaluate(model, X_train, y_train, X_test, y_test, device):
    with tf.device(device):
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time

        start_time = time.time()
        y_pred = model.predict(X_test)
        inference_time = time.time() - start_time

        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)

        return train_time, inference_time, accuracy, report['macro avg']['f1-score'], report['macro avg']['precision'], report['macro avg']['recall']

# CPU
train_time_cpu, inference_time_cpu, accuracy_cpu, f1_cpu, precision_cpu, recall_cpu = train_and_evaluate(best_model, X_train, y_train, X_test, y_test, "/CPU:0")

# GPU
train_time_gpu, inference_time_gpu, accuracy_gpu, f1_gpu, precision_gpu, recall_gpu = train_and_evaluate(best_model, X_train, y_train, X_test, y_test, "/GPU:0")

# TPU
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    train_time_tpu, inference_time_tpu, accuracy_tpu, f1_tpu, precision_tpu, recall_tpu = -1, -1, -1, -1, -1, -1  # Initialize with default values
except Exception as e:
    print("TPU Error:", e)
    train_time_tpu, inference_time_tpu, accuracy_tpu, f1_tpu, precision_tpu, recall_tpu = -1, -1, -1, -1, -1, -1

# Print results
print("CPU: Train Time =", train_time_cpu, "s, Inference Time =", inference_time_cpu, "s, Accuracy =", accuracy_cpu, ", F1 Score =", f1_cpu, ", Precision =", precision_cpu, ", Recall =", recall_cpu)
print("GPU: Train Time =", train_time_gpu, "s, Inference Time =", inference_time_gpu, "s, Accuracy =", accuracy_gpu, ", F1 Score =", f1_gpu, ", Precision =", precision_gpu, ", Recall =", recall_gpu)
print("TPU: Train Time =", train_time_tpu, "s, Inference Time =", inference_time_tpu, "s, Accuracy =", accuracy_tpu, ", F1 Score =", f1_tpu, ", Precision =", precision_tpu, ", Recall =", recall_tpu)

# Save the best model
model_filename = "/content/drive/My Drive/svm_model_best.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)


: 

In [ ]:
# Visualization: Performance comparison
labels = ['Train Time', 'Inference Time', 'Accuracy', 'F1 Score', 'Precision', 'Recall']
cpu_values = [train_time_cpu, inference_time_cpu, accuracy_cpu, f1_cpu, precision_cpu, recall_cpu]
gpu_values = [train_time_gpu, inference_time_gpu, accuracy_gpu, f1_gpu, precision_gpu, recall_gpu]
tpu_values = [train_time_tpu, inference_time_tpu, accuracy_tpu, f1_tpu, precision_tpu, recall_tpu]

x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 6))

rects1 = ax.bar(x - width, cpu_values, width, label='CPU', color='blue')
rects2 = ax.bar(x, gpu_values, width, label='GPU', color='orange')
rects3 = ax.bar(x + width, tpu_values, width, label='TPU', color='green')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()


: 

In [ ]:
# Visualization: Individual performance metrics for CPU, GPU, TPU
def plot_individual_performance(metric_name, cpu_value, gpu_value, tpu_value):
    labels = ['CPU', 'GPU', 'TPU']
    values = [cpu_value, gpu_value, tpu_value]

    plt.figure(figsize=(8, 6))
    plt.plot(labels, values, marker='o')
    plt.title(f'{metric_name} Comparison')
    plt.xlabel('Device')
    plt.ylabel(metric_name)
    plt.grid(True)
    plt.show()

# Plot individual performance metrics
plot_individual_performance('Training Time (s)', train_time_cpu, train_time_gpu, train_time_tpu)
plot_individual_performance('Inference Time (s)', inference_time_cpu, inference_time_gpu, inference_time_tpu)
plot_individual_performance('Accuracy', accuracy_cpu, accuracy_gpu, accuracy_tpu)
plot_individual_performance('F1 Score', f1_cpu, f1_gpu, f1_tpu)
plot_individual_performance('Precision', precision_cpu, precision_gpu, precision_tpu)
plot_individual_performance('Recall', recall_cpu, recall_gpu, recall_tpu)


: 

In [ ]:
import matplotlib.pyplot as plt

# Function to plot performance metrics for CPU in a pie chart
def plot_cpu_performance_pie(train_time_cpu, inference_time_cpu, accuracy_cpu, f1_cpu, precision_cpu, recall_cpu):
    labels = ['Training Time', 'Inference Time', 'Accuracy', 'F1 Score', 'Precision', 'Recall']
    values = [train_time_cpu, inference_time_cpu, accuracy_cpu, f1_cpu, precision_cpu, recall_cpu]

    plt.figure(figsize=(8, 8))
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('Performance Metrics for CPU')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

# Plot CPU performance metrics in a pie chart
plot_cpu_performance_pie(train_time_cpu, inference_time_cpu, accuracy_cpu, f1_cpu, precision_cpu, recall_cpu)


: 

In [ ]:
import matplotlib.pyplot as plt

# Function to plot performance metrics for GPU in a pie chart
def plot_gpu_performance_pie(train_time_gpu, inference_time_gpu, accuracy_gpu, f1_gpu, precision_gpu, recall_gpu):
    labels = ['Training Time', 'Inference Time', 'Accuracy', 'F1 Score', 'Precision', 'Recall']
    values = [train_time_gpu, inference_time_gpu, accuracy_gpu, f1_gpu, precision_gpu, recall_gpu]

    plt.figure(figsize=(8, 8))
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('Performance Metrics for GPU')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

# Plot GPU performance metrics in a pie chart
plot_gpu_performance_pie(train_time_gpu, inference_time_gpu, accuracy_gpu, f1_gpu, precision_gpu, recall_gpu)


: 

In [ ]:
def evaluate_model_on_input(model, scaler, device):
    # Take input from the user
    age = 45
    sex = 1
    chest_pain_type = 3
    resting_blood_pressure = 130
    cholesterol = 210
    fasting_blood_sugar = 1
    rest_ecg = 0
    max_heart_rate = 160
    exercise_induced_angina = 0
    st_depression = 1.5
    st_slope = 2
    num_major_vessels = 1
    thalassemia = 2

    # Scale the input features
    input_features = np.array([[age, sex, chest_pain_type, resting_blood_pressure, cholesterol, fasting_blood_sugar, rest_ecg,
                                 max_heart_rate, exercise_induced_angina, st_depression, st_slope, num_major_vessels, thalassemia]])
    input_features_scaled = scaler.transform(input_features)

    # Evaluate the model on the input features
    with tf.device(device):
        start_time = time.time()
        y_pred = model.predict(input_features_scaled)
        inference_time = time.time() - start_time

    print("Prediction:")
    if y_pred[0] == 0:
        print("No heart disease detected.")
    else:
        print("Heart disease detected.")

    print("Inference Time on", device, ":", inference_time, "s")

# Load the best model
with open(model_filename, 'rb') as file:
    best_model = pickle.load(file)

# Evaluate the model on input from the user
evaluate_model_on_input(best_model, scaler, "/CPU:0")
evaluate_model_on_input(best_model, scaler, "/GPU:0")
evaluate_model_on_input(best_model, scaler, resolver.master() if 'resolver' in locals() else None)


: 